In [2]:
from whatstk import WhatsAppChat
import pandas as pd

# Caminho para o arquivo de chat
filepath = '/Users/brenojaruzo/Downloads/_chat 6.txt'

# Especificando o formato do cabeçalho
hformat = '[%d/%m/%y, %H:%M:%S] %name:'

# Função para carregar e filtrar o DataFrame por período de tempo
def carregar_e_filtrar_chat(filepath, hformat, inicio=None, fim=None):
    chat = WhatsAppChat.from_source(filepath=filepath, hformat=hformat)
    df = chat.df
    
    if inicio:
        df = df[df['date'] >= pd.to_datetime(inicio)]
    if fim:
        df = df[df['date'] <= pd.to_datetime(fim)]
    
    return df

# Define o período de tempo desejado
inicio = '2024-06-18'
fim = '2024-06-19'

try:
    # Carregar e filtrar o chat
    df = carregar_e_filtrar_chat(filepath, hformat, inicio, fim)
    
    # Agrupando por 'username', contando as mensagens e ordenando por ordem decrescente de contagem
    sorted_df = df.groupby('username')['message'].count().sort_values(ascending=False)
    
    print("\nContagem de mensagens por usuário no período filtrado:")
    print(sorted_df)
    
except KeyError as e:
    print("Erro de chave:", e)
except RuntimeError as e:
    print("Erro de execução:", e)
except Exception as e:
    print("Outro erro ocorreu:", e)


Contagem de mensagens por usuário no período filtrado:
username
Pedro Almeida       326
Thiago Rodrigues    285
Victor Ghisi        140
Arthur Miranda      124
~ Bruno Longhi      121
Igor Castelo         93
Leonardo Carrera     81
Matheus Ribeiro      69
~ Victor             49
~ John Jader         39
Julio Adeodato       16
Breno j.              9
~ Caio Couto          3
~ Rafa Ribeiro        1
Name: message, dtype: int64


In [6]:
# Cria uma análise de quem mais falou, quebrado por mês.

# Certifique-se de que a coluna de data está no formato datetime
chat.df['date'] = pd.to_datetime(chat.df['date'], format='%d/%m/%y, %H:%M:%S')

# Criar uma nova coluna para o ano e o mês
chat.df['year_month'] = chat.df['date'].dt.to_period('M')

# Agrupar pelo nome do usuário e pelo mês e contar as mensagens
message_counts_by_month = chat.df.groupby(['username', 'year_month']).size().reset_index(name='message_count')

# Ordenar o resultado por nome do usuário e data
message_counts_by_month = message_counts_by_month.sort_values(by=['username', 'year_month'])

print(message_counts_by_month)

NameError: name 'chat' is not defined

In [17]:
#download de quem mais fala, por mês.
caminho_arquivo = '/Users/brenojaruzo/Downloads/message_counts_by_month2.csv'
message_counts_by_month.to_csv(caminho_arquivo, index=False)

In [27]:
from whatstk import FigureBuilder
from plotly.offline import plot
fb = FigureBuilder(chat=chat)
fig = fb.user_interventions_count_linechart(cumulative=True, msg_length=True, title='Count of sent characters (cumulative)')
plot(fig)

'temp-plot.html'

In [29]:
# Agrupar as mensagens por usuário e calcular a contagem total de caracteres em cada grupo
contagem_caracteres_por_usuario = chat.df.groupby('username')['message'].apply(lambda x: x.str.len().sum()).reset_index()

# Renomear as colunas
contagem_caracteres_por_usuario.columns = ['Usuário', 'Contagem de Caracteres']

# Ordenar a tabela pela contagem de caracteres em ordem decrescente
contagem_caracteres_por_usuario = contagem_caracteres_por_usuario.sort_values(by='Contagem de Caracteres', ascending=False)

# Resetar o índice
contagem_caracteres_por_usuario = contagem_caracteres_por_usuario.reset_index(drop=True)

# Exibir a tabela
print(contagem_caracteres_por_usuario)

             Usuário  Contagem de Caracteres
0   Thiago Rodrigues                 1397251
1      Pedro Almeida                 1278193
2       Victor Ghisi                 1201476
3       Igor Castelo                  902795
4     ~ Bruno Longhi                  723752
5   Leonardo Carrera                  491674
6     Arthur Miranda                  403371
7           Breno j.                  390479
8    Matheus Ribeiro                  321766
9     Julio Adeodato                  223494
10          ~ Victor                   94397
11      ~ John Jader                   84208
12       Victor Katz                   55552
13    ~ Rafa Ribeiro                   30337
14  ~ Pedro Monteiro                   28161
15       Luis Galvao                   22690
16      ~ Caio Couto                   14061
17   ~ Hugo Spinelli                    5428
18      Victor Alves                    1499


In [45]:
#boxplot com o tamanho médio das mensagens enviadas

fig = fb.user_msg_length_boxplot()
plot(fig)

'temp-plot.html'

In [47]:
#interação entre os usuários, enviando respostas após o outro

fig = fb.user_message_responses_heatmap()
plot(fig)

'temp-plot.html'